In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - SPAM/NOT SPAM - ITERACIÓN 6        ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#      V3 OPTIMIZED + SMART TUNING                      ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        09/12/2025  -  17:30:00           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    09/12/2025  -  17:30:00           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#      ESTRATEGIA V6 - MEJORAS SOBRE V3 (MCC 0.8733):                                              #
#      ✅ Mantener: Arquitectura simple LSTM + regularización fuerte                                #
#      🔧 Optimizar: Vocabulario (10k→20k), MAX_LEN (200→250), LSTM (64→80)                        #
#      🎯 Nuevo: Attention layer simple, threshold optimization                                     #
#      ❌ Evitar: Transfer learning, CNN híbrido, ensemble prematuro                                #
#                                                                                                    #
#      LECCIONES APRENDIDAS:                                                                        #
#      - V4 (0.64): Transfer learning fracasa en spam-specific                                     #
#      - V5 (0.81-0.86): CNN+LSTM demasiado complejo para dataset pequeño                          #
#      - V6_ens (0.82): Early stopping muy agresivo, terminó sin entrenar                          #
#                                                                                                    #
#====================================================================================================#

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

seed = 42
np.random.seed(seed)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Bidirectional, Dense, Dropout,
    GlobalMaxPooling1D, SpatialDropout1D, Attention, Permute, Multiply
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 36)
pd.set_option("display.max_colwidth", 150)

# Silent imports

## Iteración 6 - V3 Optimizado

**Estado Actual:**
- V1: MCC 0.8665 (overfitting)
- V2: MCC 0.8885 (best LSTM)
- V3: MCC 0.8733 ← BASE (regularización extrema)
- V4: MCC 0.6456 (DistilBERT fracaso)
- V5: MCC 0.81-0.86 (CNN+LSTM overfitting)
- V6_ens: MCC 0.82 (early stop prematuro)

**Estrategia V6:**
1. **Base V3**: Arquitectura simple que funciona
2. **Mejoras quirúrgicas**: 
   - Vocabulario: 10k → 20k (más información)
   - MAX_LEN: 200 → 250 (capturar más contexto)
   - LSTM: 64 → 80 units (+25% capacidad controlada)
   - Attention: Capa simple de atención post-LSTM
3. **Regularización ajustada**:
   - L2: 5e-4 → 3e-4 (menos restrictiva)
   - Dropout: 0.7 → 0.6 (permitir más info)
4. **Training mejorado**:
   - Early stopping patience: 2 → 4 (dar más tiempo)
   - LR: 5e-4 → 7e-4 (convergencia más rápida)
   - Threshold optimization post-training

**Objetivo:** MCC > 0.90

In [ ]:
# Hyperparameters - V6 = V3 CON MICRO-OPTIMIZACIONES

# Text processing - MANTENER V3
MAX_WORDS = 10000      # Mantener V3 (20k añadía ruido)
MAX_LEN = 200          # Mantener V3 (250 causaba overfitting)
EMBEDDING_DIM = 100    # Mantener V3

# Model architecture - VOLVER A V3 EXACTO
LSTM_UNITS = 64        # EXACTO V3 (72 causaba overfitting)
DENSE_UNITS = 32       # Mantener V3

# Regularization - V3 EXACTO + MÁS FUERTE
SPATIAL_DROPOUT = 0.4  # EXACTO V3
DROPOUT_RATE = 0.7     # EXACTO V3 (0.65 era poco)
L2_REG = 6e-4          # V3: 5e-4 → 6e-4 (MÁS penalización vs overfitting)

# Training - V3 BASE
BATCH_SIZE = 32
EPOCHS = 50
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 5e-4   # EXACTO V3 (6e-4 converge demasiado rápido)
CLIPNORM = 1.0

In [ ]:
# Load data
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(train['text'])

X_train_seq = tokenizer.texts_to_sequences(train['text'])
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
y_train = train['spam_label'].values

# Train/val split
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, test_size=VALIDATION_SPLIT, random_state=seed, stratify=y_train
)

## Arquitectura V6

```
Input (250)
  ↓
Embedding (20k vocab, 100 dim)
  ↓
SpatialDropout1D (0.35)
  ↓
Bi-LSTM (80 units) + L2
  ↓
Simple Attention (self-attention)
  ↓
GlobalMaxPooling1D
  ↓
Dense (32) + L2
  ↓
Dropout (0.6)
  ↓
Dense (1, sigmoid)
```

In [ ]:
def build_v6_model():
    # ARQUITECTURA V3 PURA - SIN ATTENTION (causaba overfitting)
    inputs = Input(shape=(MAX_LEN,))
    
    # Embedding
    x = Embedding(
        input_dim=MAX_WORDS,
        output_dim=EMBEDDING_DIM,
        input_length=MAX_LEN
    )(inputs)
    
    # Spatial dropout
    x = SpatialDropout1D(SPATIAL_DROPOUT)(x)
    
    # Bidirectional LSTM with regularization - SIN return_sequences
    lstm_out = Bidirectional(
        LSTM(
            LSTM_UNITS,
            kernel_regularizer=l2(L2_REG),
            recurrent_regularizer=l2(L2_REG),
            bias_regularizer=l2(L2_REG)
        )
    )(x)
    
    # SIN ATTENTION - causaba overfitting
    # Directamente a classifier
    
    # Dense classifier
    dense = Dense(
        DENSE_UNITS,
        activation='relu',
        kernel_regularizer=l2(L2_REG),
        bias_regularizer=l2(L2_REG)
    )(lstm_out)
    
    dense = Dropout(DROPOUT_RATE)(dense)
    
    # Output
    outputs = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=inputs, outputs=outputs, name='V6_LSTM_Pure_Micro_Optimized')
    return model

model = build_v6_model()

# Optimizer
optimizer = keras.optimizers.AdamW(
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    clipnorm=CLIPNORM
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()]
)

# Model built silently

In [ ]:
# Callbacks - EXACTO V3 (ultra-agresivo anti-overfitting)
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=2,  # EXACTO V3 (ultra-agresivo)
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model_v6.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=1,  # EXACTO V3 (ultra-agresivo)
        min_lr=1e-6,
        verbose=1
    )
]

history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Evaluation + DEBUG
y_pred_proba = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).flatten()

# Threshold 0.5
best_threshold = 0.5
y_pred = (y_pred_proba > best_threshold).astype(int)
mcc_val = matthews_corrcoef(y_val, y_pred)

print("="*80)
print("DEBUG - ANALYSIS")
print("="*80)
print(f"Val MCC (threshold 0.5): {mcc_val:.4f}")

# Check training history for overfitting
final_epoch = len(history.history['loss'])
train_loss_final = history.history['loss'][-1]
val_loss_final = history.history['val_loss'][-1]
train_acc_final = history.history['accuracy'][-1]
val_acc_final = history.history['val_accuracy'][-1]

overfitting_delta = val_loss_final - train_loss_final

print(f"\nTraining stopped at epoch: {final_epoch}")
print(f"Train Loss: {train_loss_final:.4f} | Val Loss: {val_loss_final:.4f}")
print(f"Train Acc:  {train_acc_final:.4f} | Val Acc:  {val_acc_final:.4f}")
print(f"Overfitting Delta: {overfitting_delta:.4f}")

if overfitting_delta > 0.15:
    print("⚠️ OVERFITTING DETECTADO (delta > 0.15)")
    print("   → Necesita MÁS regularización")
elif overfitting_delta < 0.05:
    print("✓ Sin overfitting significativo (delta < 0.05)")
    print("   → Podría permitir MENOS regularización")
else:
    print("✓ Overfitting controlado (delta 0.05-0.15)")

# Probability distribution analysis
print(f"\nProbability distribution:")
print(f"  Mean: {y_pred_proba.mean():.4f}")
print(f"  Std:  {y_pred_proba.std():.4f}")
print(f"  Min:  {y_pred_proba.min():.4f}")
print(f"  Max:  {y_pred_proba.max():.4f}")

# Predictions by class
spam_probs = y_pred_proba[y_val == 1]
notspam_probs = y_pred_proba[y_val == 0]
print(f"\nSPAM class prob mean: {spam_probs.mean():.4f} (should be > 0.5)")
print(f"NOT SPAM class prob mean: {notspam_probs.mean():.4f} (should be < 0.5)")

separation = spam_probs.mean() - notspam_probs.mean()
print(f"Class separation: {separation:.4f} (mayor = mejor)")

if separation < 0.3:
    print("⚠️ BAJA SEPARACIÓN - Modelo no distingue bien clases")
    print("   → Necesita MÁS capacidad o MEJORES features")
elif separation > 0.6:
    print("✓ BUENA SEPARACIÓN - Modelo distingue bien clases")

print("="*80)
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*80)

In [ ]:
# Test predictions
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

X_test_seq = tokenizer.texts_to_sequences(test['text'])
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

y_test_proba = model.predict(X_test_pad, batch_size=BATCH_SIZE, verbose=0).flatten()
y_test_pred = (y_test_proba > best_threshold).astype(int)

submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_test_pred
submission.to_csv('submission.csv', index=False)

print(f"\nSubmission: {len(y_test_pred)} predictions | Threshold: {best_threshold:.2f}")

In [ ]:
# Summary & Recommendations
print("\n" + "="*80)
print("RESULTADOS Y RECOMENDACIONES")
print("="*80)
print(f"Val MCC: {mcc_val:.4f} | Overfitting Δ: {overfitting_delta:.4f} | Separation: {separation:.4f}")
print("")

# Decision tree based on metrics
if overfitting_delta > 0.15:
    print("📊 DIAGNÓSTICO: OVERFITTING SEVERO")
    print("🔧 ACCIÓN:")
    print("   1. Aumentar L2: 6e-4 → 8e-4")
    print("   2. Aumentar Dropout: 0.7 → 0.75")
    print("   3. Reducir LSTM: 64 → 56 units")
    
elif overfitting_delta < 0.05 and mcc_val < 0.88:
    print("📊 DIAGNÓSTICO: UNDERFIT - Demasiada regularización")
    print("🔧 ACCIÓN:")
    print("   1. Reducir L2: 6e-4 → 4e-4")
    print("   2. Reducir Dropout: 0.7 → 0.6")
    print("   3. Aumentar LSTM: 64 → 80 units")
    
elif separation < 0.3:
    print("📊 DIAGNÓSTICO: BAJA CAPACIDAD DISCRIMINATIVA")
    print("🔧 ACCIÓN:")
    print("   1. Aumentar MAX_LEN: 200 → 220")
    print("   2. Aumentar EMBEDDING_DIM: 100 → 128")
    print("   3. Mantener regularización actual")
    
elif mcc_val >= 0.87 and overfitting_delta <= 0.12:
    print("📊 DIAGNÓSTICO: MODELO BALANCEADO")
    print("✅ ACCIÓN:")
    print("   1. Este es probablemente el mejor resultado alcanzable")
    print("   2. Usar ESTE modelo para submission")
    print("   3. Si test MCC < val MCC: problema es dataset, no modelo")

else:
    print("📊 DIAGNÓSTICO: AJUSTE FINO NECESARIO")
    print("🔧 ACCIÓN:")
    print("   1. Probar L2 entre 5e-4 y 7e-4")
    print("   2. Considerar LSTM 68-72 units")
    print("   3. Mantener threshold 0.5")

print("="*80)
print("\nHISTÓRICO:")
print("V2: 0.8885 (overfitting Δ~0.16) | V3: 0.8733 (overfitting Δ<0.08)")
print(f"V6: {mcc_val:.4f} (overfitting Δ={overfitting_delta:.4f})")
print("="*80)